## GSAT trend patterns

In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.apply_mannkendall(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [ ]:
# Input the observational trend
interval_name = ['10yr', '30yr', '60yr']

# Input the Observational internal trend (wrt MMEM GSAT)
dir_internal_input = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure3/data/ICV_std/'

HadCRUT5_annual_internal_trend_da = {}
# HadCRUT5_annual_internal_p_value_da = {}

for interval in interval_name:
    HadCRUT5_annual_internal_trend_da[interval] = xr.open_dataset(dir_internal_input + 'ICV_segments_' + interval + '_std_trend_pattern.nc')
    # HadCRUT5_annual_internal_p_value_da[interval] = xr.open_dataset(dir_internal_input + 'HadCRUT5_annual_internal_' + interval + '_p_value.nc')

In [ ]:
HadCRUT5_annual_internal_trend_da

In [ ]:
# Input the MMEM annual trend
dir_model_in = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure3/data/'
MMEM_annual_trend_da = {}
for interval in interval_name:
    MMEM_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'MMEM_annual_' + interval + '_noise_trend_std.nc')
    
# CanESM5
CanESM5_annual_trend_da = {}
for interval in interval_name:
    CanESM5_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'CanESM5_annual_' + interval + '_noise_trend_std.nc')

# IPSL
IPSL_annual_trend_da = {}
for interval in interval_name:
    IPSL_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'IPSL_annual_' + interval + '_noise_trend_std.nc')

# EC-Earth3
EC_Earth3_annual_trend_da = {}
for interval in interval_name:
    EC_Earth3_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'EC_Earth_annual_' + interval + '_noise_trend_std.nc')
   
# ACCESS
ACCESS_annual_trend_da = {}
for interval in interval_name:
    ACCESS_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'ACCESS_annual_' + interval + '_noise_trend_std.nc')
   
# MPI-ESM1-2
MPI_ESM_annual_trend_da = {}
for interval in interval_name:
    MPI_ESM_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'MPI_ESM_annual_' + interval + '_noise_trend_std.nc')
   
# MIROC6
MIROC6_annual_trend_da = {}
for interval in interval_name:
    MIROC6_annual_trend_da[interval] = xr.open_dataset(dir_model_in + 'MIROC6_annual_' + interval + '_noise_trend_std.nc')
   

In [ ]:
MMEM_annual_trend_da['10yr']

In [ ]:
CanESM5_annual_trend_da['10yr']

In [ ]:
# loop for the rest of the data
for interval in interval_name:
    # HadCRUT5
    HadCRUT5_annual_internal_trend_da[interval] = HadCRUT5_annual_internal_trend_da[interval].rename({'ICV_segments_'+interval+'_std_trend_pattern': 'tas'})
    # HadCRUT5_annual_internal_p_value_da[interval] = HadCRUT5_annual_internal_p_value_da[interval].rename({'__xarray_dataarray_variable__': 'p_value'})
    

In [ ]:
CanESM5_annual_trend_da['10yr'].tas.min().values

In [ ]:
# check the min and max value of the trend
# HadCRUT5
for interval in interval_name:
    print('HadCRUT5', interval, 'min:', HadCRUT5_annual_internal_trend_da[interval].tas.min().values, 'max:', HadCRUT5_annual_internal_trend_da[interval].tas.max().values)

### Plotting with the Robinson Projections

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.transparent'] = True

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

def plot_trend_with_significance(trend_data, lats, lons, p_values, levels=None, extend=None, cmap=None, 
                                 title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """

    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()
  
    # Determine significance mask (where p-values are less than 0.10)
    insignificance_mask = p_values >= 0.10
    
    # Plotting
    # contour_obj = ax.pcolormesh(lons, lats, trend_data,  cmap='RdBu_r',vmin=-5.0, vmax=5.0, transform=ccrs.PlateCarree(central_longitude=180), shading='auto')
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    # Plot significance masks with different hatches
    ax.contourf(lons, lats, insignificance_mask, levels=[0, 0.10, 1.0],hatches=[None,'///'], colors='none', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      color='gray', alpha=0.35, linestyle='--')

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=20, fontweight='bold', pad=5.0)

    return contour_obj


In [ ]:
def plot_trend(trend_data, lats, lons, levels=None, extend=None, cmap=None, 
                                 title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """

    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()
  
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      color='gray', alpha=0.35, linestyle='--')

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 18}
    gl.ylabel_style = {'size': 18}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj

In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

cmap=mcolors.ListedColormap(palettable.cmocean.diverging.Balance_20.mpl_colors)

### Plot the Original, internal, MMEM trend patterns

In [ ]:
HadCRUT5_annual_internal_trend_da['10yr']

In [ ]:
# arange data into a list arrocding to the variable name
trend_10yr = {"HadCRUT5":HadCRUT5_annual_internal_trend_da['10yr'], 
            "MMEM":MMEM_annual_trend_da['10yr'],
            "MIROC6":MIROC6_annual_trend_da['10yr'],
            "MPI_ESM":MPI_ESM_annual_trend_da['10yr'],
            "ACCESS":ACCESS_annual_trend_da['10yr'],
            "EC_Earth3":EC_Earth3_annual_trend_da['10yr'],
            "IPSL":IPSL_annual_trend_da['10yr'],
            "CanESM5":CanESM5_annual_trend_da['10yr']  
            }

trend_30yr = {"HadCRUT5":HadCRUT5_annual_internal_trend_da['30yr'],
            "MMEM":MMEM_annual_trend_da['30yr'],
            "MIROC6":MIROC6_annual_trend_da['30yr'],
            "MPI_ESM":MPI_ESM_annual_trend_da['30yr'],
            "ACCESS":ACCESS_annual_trend_da['30yr'],
            "EC_Earth3":EC_Earth3_annual_trend_da['30yr'],
            "IPSL":IPSL_annual_trend_da['30yr'],
            "CanESM5":CanESM5_annual_trend_da['30yr']
            }

trend_60yr = {"HadCRUT5":HadCRUT5_annual_internal_trend_da['60yr'], 
            "MMEM":MMEM_annual_trend_da['60yr'],
            "MIROC6":MIROC6_annual_trend_da['60yr'],
            "MPI_ESM":MPI_ESM_annual_trend_da['60yr'],
            "ACCESS":ACCESS_annual_trend_da['60yr'], 
            "EC_Earth3":EC_Earth3_annual_trend_da['60yr'],
            "IPSL":IPSL_annual_trend_da['60yr'],
            "CanESM5":CanESM5_annual_trend_da['60yr']
            }


In [ ]:
trend_60yr

In [ ]:
# reattribute the units for the trend
variable_name = ["HadCRUT5", "MMEM", "MIROC6", "MPI_ESM", "ACCESS", "EC_Earth3", "IPSL", "CanESM5"]
# degC/60yr, 30yr, 10yr
# for i in range(len(variable_name)):
#     trend_60yr[variable_name[i]] = trend_60yr[variable_name[i]]
#     trend_30yr[variable_name[i]] = trend_30yr[variable_name[i]]
#     trend_10yr[variable_name[i]] = trend_10yr[variable_name[i]]

In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

In [ ]:
trend_10yr['HadCRUT5']['tas'].shape

In [ ]:
# check the min and max of the trend
for i in range(len(variable_name)):
    print(trend_10yr[variable_name[i]]['tas'].min().values, trend_10yr[variable_name[i]]['tas'].max().values)
    print(trend_30yr[variable_name[i]]['tas'].min().values, trend_30yr[variable_name[i]]['tas'].max().values)
    print(trend_60yr[variable_name[i]]['tas'].min().values, trend_60yr[variable_name[i]]['tas'].max().values)

In [ ]:
# pattern correlation betwenn observed internal pattern vs. Model simulated internal pattern
import scipy.stats as stats

trend_pattern_correlation_10yr = []

for i in range(len(variable_name)):
    trend_pattern_correlation_10yr.append(stats.pearsonr(trend_10yr['HadCRUT5']['tas'].values.flatten(), trend_10yr[variable_name[i]]['tas'].values.flatten())[0])

trend_pattern_correlation_10yr 


In [ ]:
trend_pattern_correlation_30yr = []
for i in range(len(variable_name)):
    trend_pattern_correlation_30yr.append(stats.pearsonr(trend_30yr['HadCRUT5']['tas'].values.flatten(), trend_30yr[variable_name[i]]['tas'].values.flatten())[0])

trend_pattern_correlation_30yr

trend_pattern_correlation_60yr = []
for i in range(len(variable_name)):
    trend_pattern_correlation_60yr.append(stats.pearsonr(trend_60yr['HadCRUT5']['tas'].values.flatten(), trend_60yr[variable_name[i]]['tas'].values.flatten())[0])
print(trend_pattern_correlation_60yr)

In [ ]:
# trend_pattern_correlation_20yr = []
# for i in range(len(variable_name)):
#     trend_pattern_correlation_20yr.append(stats.pearsonr(trend_20yr['HadCRUT5']['tas'].values.flatten(), trend_20yr[variable_name[i]]['tas'].values.flatten())[0])
# print(trend_pattern_correlation_20yr)
# trend_pattern_correlation_40yr = []
# for i in range(len(variable_name)):
#     trend_pattern_correlation_40yr.append(stats.pearsonr(trend_40yr['HadCRUT5']['tas'].values.flatten(), trend_40yr[variable_name[i]]['tas'].values.flatten())[0])

# trend_pattern_correlation_40yr

# trend_pattern_correlation_50yr = []
# for i in range(len(variable_name)):
#     trend_pattern_correlation_50yr.append(stats.pearsonr(trend_50yr['HadCRUT5']['tas'].values.flatten(), trend_50yr[variable_name[i]]['tas'].values.flatten())[0])

# trend_pattern_correlation_50yr

# trend_pattern_correlation_70yr = []
# for i in range(len(variable_name)):
#     trend_pattern_correlation_70yr.append(stats.pearsonr(trend_70yr['HadCRUT5']['tas'].values.flatten(), trend_70yr[variable_name[i]]['tas'].values.flatten())[0])

In [ ]:
# save the correlation into one dictionary
trend_pattern_correlation = {'10yr': trend_pattern_correlation_10yr, '30yr': trend_pattern_correlation_30yr, '60yr': trend_pattern_correlation_60yr}

In [ ]:
trend_pattern_correlation

In [ ]:
HadCRUT5_annual_internal_trend_da['10yr']

In [ ]:
# obs_trend = xr.Dataset({'10yr': HadCRUT5_annual_internal_trend_da['10yr']['tas'], 
#                         '30yr': HadCRUT5_annual_internal_trend_da['30yr']['tas'],
#                         '60yr': HadCRUT5_annual_internal_trend_da['60yr']['tas']})
# p_val_data = xr.Dataset({'10yr': HadCRUT5_annual_internal_p_value_da['10yr']['p_value'],
#                             '30yr': HadCRUT5_annual_internal_p_value_da['30yr']['p_value'],
#                             '60yr': HadCRUT5_annual_internal_p_value_da['60yr']['p_value']})

In [ ]:
# obs_trend

In [ ]:
# # save the pattern correlation from the second to the last, which corresponds to the MMEM, CanESM5, IPSL, EC-Earth3, ACCESS, MPI-ESM, MIROC6
# with open('pattern_correlations_noise_trend_std_model_vs_Obs.txt', 'w') as file:
#     file.write('10-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_10yr:
#         file.write(f"{correlation}\n")

#     file.write('\n30-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_30yr:
#         file.write(f"{correlation}\n")

#     file.write('\n60-year Trend Pattern Correlations:\n')
#     for correlation in trend_pattern_correlation_60yr:
#         file.write(f"{correlation}\n")


In [ ]:
# Plotting
lat = trend_60yr['HadCRUT5'].lat
lon = trend_60yr['HadCRUT5'].lon
lat, lon 

titles_rows = ["HadCRUT5", "MMEM", "MIROC6", "MPI-ESM1.2-LR","ACCESS-ESM1.5", "EC-Earth3", "IPSL-CM6A-LR", "CanESM5"]
rows_label = ["a", "b", "c", "d", "e", "f", "g", "h"]
titles_columns = ["2013-2022 (10yr)", "1993-2022 (30yr)", "1963-2022 (60yr)"]

import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

periods = ["10yr", "30yr", "60yr"]
variable_name = ["HadCRUT5","MMEM", "MIROC6", "MPI_ESM", "ACCESS", "EC_Earth3", "IPSL", "CanESM5"]

intervals_obs = np.arange(-1.0, 1.1, 0.1)
intervals = np.arange(0.0, 1.05, 0.05)

cmap = mcolors.ListedColormap(palettable.cmocean.sequential.Amp_20.mpl_colors)
extend = 'both'

fig  = plt.figure(figsize=(20, 30))
gs = gridspec.GridSpec(8, 3, figure=fig,wspace=0.05, hspace=0.2)

# Create a 8x3 grid of subplots
axes = {}

# for i in range(0, 1):
#     # Add cyclic points
#     for j, period in enumerate(periods):
#         print(period)
#         is_left = j == 0
#         is_bottom_row = i >= 7
        
#         ax = plt.subplot(gs[i, j], projection=ccrs.Robinson(180))
#         ax.set_global()
#         axes[i, j] = ax
#         trend_data = obs_trend[period]
#         pval_data = p_val_data[period]
#         trend_with_cyclic, lons_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
#         p_value_with_cyclic, _ = cutil.add_cyclic_point(pval_data, coord=lon)
            
#         contour_obj = plot_trend_with_significance(trend_with_cyclic, lat, lons_cyclic,p_value_with_cyclic, levels=intervals_obs, extend = extend,
#                                             cmap='twilight_shifted', title=" ", ax=ax,
#                                             show_xticks = False, show_yticks = is_left)
for i in np.arange(0,8,1):
    for j, period in enumerate(periods):
        is_left = j == 0
        is_bottom_row = i >= 7
        
        ax = plt.subplot(gs[i, j], projection=ccrs.Robinson(180))
        ax.set_global()
        axes[i, j] = ax
                        
        if j == 0:
        # Add cyclic points
            trend_data = trend_10yr[variable_name[i]]['tas']
            trend_with_cyclic, lons_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
                    
            contour_obj1 = plot_trend(trend_with_cyclic, lat, lons_cyclic, 
                                            levels=intervals, extend = 'max',
                                            cmap=cmap, title=" ", ax=ax, 
                                            show_xticks = is_bottom_row, show_yticks = is_left)
        elif j == 1:
            trend_data = trend_30yr[variable_name[i]]['tas']
            trend_with_cyclic, lons_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
                    
            contour_obj2 = plot_trend(trend_with_cyclic, lat, lons_cyclic, 
                                            levels=intervals, extend = 'max',
                                            cmap=cmap, title=" ", ax=ax, 
                                            show_xticks = is_bottom_row, show_yticks = is_left)
        else:
            trend_data = trend_60yr[variable_name[i]]['tas']
            trend_with_cyclic, lons_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
                    
            contour_obj3 = plot_trend(trend_with_cyclic, lat, lons_cyclic,  
                                            levels=intervals, extend = 'max',
                                            cmap=cmap, title=" ", ax=ax, 
                                            show_xticks = is_bottom_row, show_yticks = is_left)

# add the title for each row
for i, var in enumerate(variable_name):
    axes[i, 0].text(-0.2, 0.5, titles_rows[i], va='center', ha='center', rotation=90, 
                    fontsize=20,transform=axes[i, 0].transAxes)
    axes[i, 0].text(-0.08, 1.05, rows_label[i], va='bottom', ha='right', rotation='horizontal',
                    fontsize=24, fontweight='bold', transform=axes[i, 0].transAxes)

# add the title for each column
for j in range(3):
    axes[0,j].text(0.5, 1.05, titles_columns[j], va='bottom', ha='center', rotation='horizontal', 
                   fontsize=22,transform=axes[0, j].transAxes)

# Add the pattern correlation text to the second to last rows of each column subplot
for i in np.arange(1,8,1):
    axes[i, 0].text(0.65, 1.0, f"corr: {trend_pattern_correlation_10yr[i]:.2f}", va='bottom', ha='left', fontsize=22,transform=axes[i, 0].transAxes)
    axes[i, 1].text(0.65, 1.0, f"corr: {trend_pattern_correlation_30yr[i]:.2f}", va='bottom', ha='left', fontsize=22,transform=axes[i, 1].transAxes)
    axes[i, 2].text(0.65, 1.0, f"corr: {trend_pattern_correlation_60yr[i]:.2f}", va='bottom', ha='left', fontsize=22,transform=axes[i, 2].transAxes)

# Add horizontal colorbars
# cbar_ax = fig.add_axes([0.2, 0.08, 0.3, 0.012])
# cbar = plt.colorbar(contour_obj, cax=cbar_ax, orientation='horizontal', extend=extend)
# cbar.ax.tick_params(labelsize=16)
# cbar.set_label('Annual SAT Trend (°C/decade)', fontsize=16)

cbar_ax1 = fig.add_axes([0.25, 0.08, 0.5, 0.01])
cbar1 = plt.colorbar(contour_obj1, cax=cbar_ax1, orientation='horizontal', extend='max')
cbar1.ax.tick_params(labelsize=18)
cbar1.set_label('SAT trend Stddev.(°C/decade)', fontsize=20)

# plt.figure(constrained_layout=True)
fig.savefig('Extended-Fig-3-check.png', dpi=300, bbox_inches='tight')
fig.savefig('Extended-Fig-3-check.eps', format='eps', dpi=300, bbox_inches='tight')
fig.savefig('Extended-Fig-3-check.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()